In [243]:
import random
import numpy as np
import tensorflow.compat.v1 as tf
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras import layers, Sequential, Input
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.layers import Dense,LSTM, Concatenate, Input, Conv1D, Flatten, Dropout, BatchNormalization, GlobalAveragePooling1D, MaxPooling1D, GlobalAveragePooling1D, UpSampling1D, concatenate, Conv1DTranspose, Cropping1D, UpSampling1D, ZeroPadding1D, Reshape, Lambda, Multiply, Softmax
from keras.models import Sequential,Model
matplotlib.use('TkAgg')

In [447]:
data = pd.read_csv("data/DataBase_RF_bi_lowperc_1.csv")

In [448]:
data

,name,Al,Si,Cation #1,Cation #2,Radius #1,Radius #2,Polarizability #1,Polarizability #2,Charge #1,Charge #2,S1 #1,S1' #1,S2 #1,S3 #1,S1 #2,S1' #2,S2 #2,S3 #2
0,Structure: Al_80_Si_112_Boulfelfel_Ca_1_K_78,80,112,1,78,1.00,1.38,22.80,43.40,2,1,0,0,1,0,16,7,34,21
1,Structure: Al_80_Si_112_Boulfelfel_Ca_1_Na_78,80,112,1,78,1.00,1.02,22.80,24.11,2,1,0,0,1,0,1,25,30,22
2,Structure: Al_80_Si_112_Boulfelfel_Ca_1_Rb_78,80,112,1,78,1.00,1.52,22.80,47.30,2,1,1,0,0,0,14,8,35,21
3,Structure: Al_80_Si_112_Boulfelfel_Cs_1_K_79,80,112,1,79,1.67,1.38,59.42,43.40,1,1,0,0,1,0,16,8,35,20
4,Structure: Al_80_Si_112_Boulfelfel_Cs_1_Li_79,80,112,1,79,1.67,0.76,59.42,24.33,1,1,0,0,0,1,0,23,31,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Structure: Al_96_Si_96_Boulfelfel_Rb_14_Na_82,96,96,14,82,1.52,1.02,47.30,24.11,1,1,0,0,0,14,0,32,32,18
496,Structure: Al_96_Si_96_Boulfelfel_Rb_15_Na_81,96,96,15,81,1.52,1.02,47.30,24.11,1,1,0,0,1,14,0,32,31,18
497,Structure: Al_96_Si_96_Boulfelfel_Rb_17_K_79,96,96,17,79,1.52,1.38,47.30,43.40,1,1,2,2,4,9,14,5,35,25
498,Structure: Al_96_Si_96_Boulfelfel_Rb_4_Li_92,96,96,4,92,1.52,0.76,47.30,24.33,1,1,0,0,1,3,0,32,31,29


In [449]:
data.iloc[22]

name                 Structure: Al_81_Si_111_Boulfelfel_Cs_2_Li_79
Al                                                              81
Si                                                             111
Cation #1                                                        2
Cation #2                                                       79
Radius #1                                                     1.67
Radius #2                                                     0.76
Polarizability #1                                            59.42
Polarizability #2                                            24.33
Charge #1                                                        1
Charge #2                                                        1
S1 #1                                                            0
S1' #1                                                           0
S2 #1                                                            0
S3 #1                                                         

In [451]:
data.loc[(data["Cation #1"] + data["Cation #2"]) != (data["S1 #1"] + data["S1\' #1"] + data["S2 #1"]+ data["S3 #1"]+ data["S1 #2"]+ data["S1\' #2"]+ data["S2 #2"]+ data["S3 #2"])]["name"]

22      Structure: Al_81_Si_111_Boulfelfel_Cs_2_Li_79
30      Structure: Al_81_Si_111_Boulfelfel_Li_1_Rb_80
47       Structure: Al_82_Si_110_Boulfelfel_Cs_3_K_79
68       Structure: Al_83_Si_109_Boulfelfel_Cs_2_K_81
70       Structure: Al_83_Si_109_Boulfelfel_Cs_3_K_80
106      Structure: Al_84_Si_108_Boulfelfel_Cs_5_K_79
138      Structure: Al_85_Si_107_Boulfelfel_K_4_Cs_81
160      Structure: Al_86_Si_106_Boulfelfel_Cs_1_K_85
194      Structure: Al_87_Si_105_Boulfelfel_Cs_6_K_81
225      Structure: Al_88_Si_104_Boulfelfel_Cs_9_K_79
256     Structure: Al_88_Si_104_Boulfelfel_Rb_84_Ca_2
296    Structure: Al_90_Si_102_Boulfelfel_Na_11_Cs_79
344      Structure: Al_92_Si_100_Boulfelfel_Cs_8_K_84
356     Structure: Al_92_Si_100_Boulfelfel_Na_2_Cs_90
360     Structure: Al_92_Si_100_Boulfelfel_Na_9_Cs_83
398      Structure: Al_93_Si_99_Boulfelfel_Rb_8_Cs_85
405      Structure: Al_94_Si_98_Boulfelfel_Ca_6_Rb_82
Name: name, dtype: object

In [408]:
data_without_names = data.drop(columns=['name'])
plt.matshow(data_without_names.corr())
continuous_features = data_without_names.describe().columns
plt.xticks(range(len(continuous_features)), continuous_features)
plt.yticks(range(len(continuous_features)), continuous_features)
plt.colorbar()
plt.show()

In [547]:
features = data.drop(columns=["name", "S1 #1", "S1\' #1", "S2 #1", "S3 #1", "S1 #2", "S1\' #2", "S2 #2", "S3 #2"], axis=1)
labels = data[["S1 #1" ,"S1\' #1", "S2 #1", "S3 #1", "S1 #2", "S1\' #2", "S2 #2", "S3 #2"]]

In [593]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.25, random_state = 0)

In [594]:
features_train

,Al,Si,Cation #1,Cation #2,Radius #1,Radius #2,Polarizability #1,Polarizability #2,Charge #1,Charge #2
239,88,104,1,87,1.02,1.67,24.11,59.42,1,1
59,82,110,80,1,1.02,1.00,24.11,22.80,1,2
406,94,98,7,80,1.00,1.52,22.80,47.30,2,1
391,93,99,2,91,1.02,1.38,24.11,43.40,1,1
411,94,98,92,1,1.67,1.00,59.42,22.80,1,2
...,...,...,...,...,...,...,...,...,...,...
323,91,101,6,85,1.02,1.38,24.11,43.40,1,1
192,87,105,2,85,1.67,1.52,59.42,47.30,1,1
117,84,108,78,3,0.76,1.00,24.33,22.80,1,2
47,82,110,3,79,1.67,1.38,59.42,43.40,1,1


In [222]:
from sklearn.preprocessing import MinMaxScaler
# scale_features_mm = MinMaxScaler()
# features_train = scale_features_mm.fit_transform(features_train)
# features_test = scale_features_mm.transform(features_test)

scale_labels_mm = MinMaxScaler()
labels_train = scale_labels_mm.fit_transform(labels_train)
labels_test = scale_labels_mm.transform(labels_test)

In [383]:
features_train

,Al,Si,Cation #1,Cation #2,Radius #1,Radius #2,Polarizability #1,Polarizability #2,Charge #1,Charge #2
454,95,97,9,86,0.76,1.38,24.33,43.40,1,1
416,94,98,2,92,0.76,1.52,24.33,47.30,1,1
167,86,106,3,83,1.38,1.52,43.40,47.30,1,1
165,86,106,3,83,1.38,1.67,43.40,59.42,1,1
52,82,110,3,79,1.38,1.52,43.40,47.30,1,1
...,...,...,...,...,...,...,...,...,...,...
335,92,100,12,80,1.67,0.76,59.42,24.33,1,1
201,87,105,2,85,0.76,1.38,24.33,43.40,1,1
123,84,108,5,79,1.02,1.67,24.11,59.42,1,1
50,82,110,3,79,1.38,0.76,43.40,24.33,1,1


In [703]:
def mps_loss(y_true, y_pred):
    # Приводим y_true и y_pred к типу float32, чтобы избежать несоответствия типов
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    
    # Вычисление абсолютных отклонений
    delta = tf.abs(y_true - y_pred)
    
    # Среднее значение отклонений
    delta_ = tf.reduce_mean(delta)
    
    # Стандартное отклонение отклонений
    sigma = tf.sqrt(tf.reduce_mean(tf.square(delta - delta_)))
    
    # Получаем размер тензора в целочисленном типе (int32)
    size = tf.size(delta, out_type=tf.int32)
    
    # Финальная ошибка
    return tf.reduce_mean(delta_ / tf.cast(size, tf.float32) * (sigma / tf.cast(size, tf.float32) + 1))

def create_model(x):
    inputs = Input(shape=10)
    num_elements = Input(shape=1)
    x = Dense(10, activation='relu')(inputs)
    x = Dropout(0.3)(x)
    d = Dense(8, activation='relu')(x)
    l1 = Lambda(lambda x: x / tf.reduce_sum(x, axis=1, keepdims=True))(d)
    outputs = Multiply()([l1, num_elements])
    model = tf.keras.Model(inputs=[inputs, num_elements], outputs=outputs)
    model.compile(loss=mps_loss, optimizer='Adam')
    return model

model = create_model(features_train)

In [603]:
num_train = []
num_test = []

for i in range(len(features_train)):
    #labels_train.iloc[i] /= np.sum(labels_train.iloc[i])
    num_train.append(features_train.iloc[i]["Al"])
for i in range(len(features_test)):
    #labels_test.iloc[i] /= np.sum(labels_test.iloc[i])
    num_test.append(features_test.iloc[i]["Al"])

In [704]:
def plot():
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_loss(history):
    plt.plot(history.history['loss'], label='loss', marker=".")
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Error [MPG]')
    plot()

mcp_save = tf.keras.callbacks.ModelCheckpoint(filepath='data/models/mps.keras', save_best_only=True,
                                              monitor='val_loss', mode='min', verbose=1, save_weights_only=False,
                                              save_freq='epoch')

history = model.fit([features_train, np.array(num_train)], labels_train,
                    epochs=100, batch_size=16, shuffle=True,
                    validation_data=([features_test, np.array(num_test)], labels_test), callbacks=[mcp_save])
plot_loss(history)

Epoch 1/100
23/24 [===========================>..] - ETA: 0s - loss: 0.1016
Epoch 1: val_loss improved from inf to 0.09219, saving model to data/models\mps.keras
24/24 [==============================] - 1s 13ms/step - loss: 0.1039 - val_loss: 0.0922
Epoch 2/100
23/24 [===========================>..] - ETA: 0s - loss: 0.0861
Epoch 2: val_loss improved from 0.09219 to 0.07870, saving model to data/models\mps.keras
24/24 [==============================] - 0s 7ms/step - loss: 0.0881 - val_loss: 0.0787
Epoch 3/100
13/24 [===============>..............] - ETA: 0s - loss: 0.0784
Epoch 3: val_loss improved from 0.07870 to 0.06710, saving model to data/models\mps.keras
24/24 [==============================] - 0s 7ms/step - loss: 0.0780 - val_loss: 0.0671
Epoch 4/100
13/24 [===============>..............] - ETA: 0s - loss: 0.0681
Epoch 4: val_loss improved from 0.06710 to 0.05455, saving model to data/models\mps.keras
24/24 [==============================] - 0s 7ms/step - loss: 0.0679 - val_loss

In [671]:
model = tf.keras.models.load_model('data/models/mps.keras')

ValueError: Unknown loss function: mps_loss. Please ensure this object is passed to the `custom_objects` argument. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [705]:
predictions = model.predict([features_test, np.array(num_test)])

4/4 [==============================] - 0s 3ms/step


In [706]:
features_test.iloc[88]

Al                    92.00
Si                   100.00
Cation #1             88.00
Cation #2              2.00
Radius #1              1.38
Radius #2              1.00
Polarizability #1     43.40
Polarizability #2     22.80
Charge #1              1.00
Charge #2              2.00
Name: 348, dtype: float64

In [719]:
import random

def mps(first, second):
    delta = []
    for i in range(len(first)):
        delta.append(abs(first[i] - second[i]))
    delta_ = sum(delta) / len(delta)
    sigma = 0
    for i in range(len(first)):
        sigma += (delta_ - abs(first[i] - second[i]))**2
    sigma = np.sqrt(sigma/len(delta))
    return delta_/len(delta)*(sigma/len(delta) + 1)
    
i = random.randint(0, len(predictions)-1)
print("№", i)
rounded_prediction = [round(p, 1) for p in predictions[i]]
print(rounded_prediction)

rounded_real= [round(float(p), 1) for p in np.array(labels_test.iloc[i])]
print(rounded_real)
print(sum(predictions[i]), sum(labels_test.iloc[i]))

№ 61
[0.0, 0.0, 2.7, 0.0, 14.5, 7.0, 36.5, 29.4]
[1.0, 0.0, 1.0, 0.0, 14.0, 1.0, 38.0, 33.0]
90.00000476837158 88


In [708]:
mps_list = []
for i in range(len(predictions)):
    p = predictions[i]
    real = np.array(labels_test.iloc[i])
    mps_list.append(mps(p, real))
print(sum(mps_list))

40.790410466496716


In [716]:
plt.hist(mps_list)
plt.show()

In [610]:
print(sum(mps_list))

37.59565462806723
